In [1]:
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.layers import GlobalAveragePooling2D, Flatten, Dense, Input, Dropout
from keras.models import Model, load_model
from keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator


#path = '/home/ubuntu/nbs/state_farm/data/sample1/'
#batch_size=1

path = '/home/ubuntu/nbs/state_farm/data/sample/'
#path = '/home/ubuntu/nbs/state_farm/data/'
batch_size = 64

target_size=(224, 224)

nb_train_samples = 800
nb_validation_samples = 300
epochs = 50



#generate the batches
def get_batches(directory, target_size=target_size, batch_size=batch_size, shuffle=False):
    datagen = ImageDataGenerator()
    return datagen.flow_from_directory(directory=directory,
                                          target_size=target_size,
                                          batch_size=batch_size,
                                          class_mode='categorical',
                                          shuffle=shuffle)


batches = get_batches(path+'train_set', batch_size=batch_size, shuffle=True)

valid_batches = get_batches(path+'valid_set', batch_size=batch_size*2, shuffle=False)

# initialize the model
initial_model = VGG16(weights="imagenet", include_top=True)

#finetuning
x = Dense(batches.num_class, activation='softmax')(initial_model.layers[-2].output)
model = Model(initial_model.input, x)

# we freeze the other layers 
for layer in initial_model.layers: layer.trainable=False

opt = Adam(lr=0.001)

model.compile(optimizer=opt,
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit_generator(batches, epochs=10, 
                    steps_per_epoch=nb_train_samples // batch_size,
                    validation_data=valid_batches, 
                    validation_steps=nb_validation_samples // batch_size)

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


Found 800 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
Epoch 1/10
12/12 [==============================] - 42s - loss: 2.9181 - acc: 0.1862 - val_loss: 2.5726 - val_acc: 0.1706
Epoch 2/10
12/12 [==============================] - 37s - loss: 1.5679 - acc: 0.4917 - val_loss: 2.6039 - val_acc: 0.2529
Epoch 3/10
12/12 [==============================] - 40s - loss: 0.9102 - acc: 0.7190 - val_loss: 2.3791 - val_acc: 0.3014
Epoch 4/10
12/12 [==============================] - 43s - loss: 0.6281 - acc: 0.8140 - val_loss: 2.2589 - val_acc: 0.3248
Epoch 5/10
12/12 [==============================] - 37s - loss: 0.4154 - acc: 0.9166 - val_loss: 2.3634 - val_acc: 0.3576
Epoch 6/10
12/12 [==============================] - 40s - loss: 0.3574 - acc: 0.9285 - val_loss: 2.2946 - val_acc: 0.3855
Epoch 7/10
12/12 [==============================] - 40s - loss: 0.2408 - acc: 0.9610 - val_loss: 2.1434 - val_acc: 0.4276
Epoch 8/10
12/12 [==============================] - 33s - loss

In [2]:
for layer in model.layers[:10]: 
    layer.trainable = False 

for layer in model.layers[10:]: 
    layer.trainable = True
    
opt = SGD(lr=10e-5) 
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(batches, epochs=10, 
                    steps_per_epoch=nb_train_samples // batch_size,
                    validation_data=valid_batches, 
                    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
12/12 [==============================] - 38s - loss: 0.0926 - acc: 0.9987 - val_loss: 2.3977 - val_acc: 0.3924
Epoch 2/10
12/12 [==============================] - 38s - loss: 0.0673 - acc: 0.9962 - val_loss: 2.4637 - val_acc: 0.3879
Epoch 3/10
12/12 [==============================] - 38s - loss: 0.0547 - acc: 1.0000 - val_loss: 2.4296 - val_acc: 0.4299
Epoch 4/10
12/12 [==============================] - 37s - loss: 0.0433 - acc: 1.0000 - val_loss: 2.5761 - val_acc: 0.3895
Epoch 5/10
12/12 [==============================] - 39s - loss: 0.0406 - acc: 1.0000 - val_loss: 2.6022 - val_acc: 0.4042
Epoch 6/10
12/12 [==============================] - 36s - loss: 0.0353 - acc: 1.0000 - val_loss: 2.6488 - val_acc: 0.4012
Epoch 7/10
12/12 [==============================] - 38s - loss: 0.0288 - acc: 1.0000 - val_loss: 2.6454 - val_acc: 0.4042
Epoch 8/10
12/12 [==============================] - 38s - loss: 0.0268 - acc: 1.0000 - val_loss: 2.5890 - val_acc: 0.4509
Epoch 9/10
12/12 [======

In [3]:
# predictions on new data
test_batches = get_batches(path+'test', batch_size=batch_size)
preds = model.predict_generator(test_batches, steps=test_batches.samples)


Found 10 images belonging to 1 classes.


In [4]:
preds_clipes = preds.clip(min=0.01, max=0.99)
print(preds_clipes[:10])

[[ 0.01        0.12523724  0.30430737  0.01        0.06009294  0.01
   0.0436891   0.46191174  0.01        0.01      ]
 [ 0.01        0.01        0.64018762  0.01        0.01        0.01
   0.3480444   0.01        0.01        0.01      ]
 [ 0.01        0.01        0.01        0.01        0.01        0.01        0.01
   0.01        0.01        0.99000001]
 [ 0.01        0.03554508  0.50475109  0.01        0.35027114  0.01
   0.03962713  0.01        0.02117522  0.04019851]
 [ 0.01        0.02392313  0.08448274  0.01        0.05023948  0.01651551
   0.01        0.76154894  0.01        0.05397846]
 [ 0.01        0.01        0.01        0.01        0.01        0.52133161
   0.12427779  0.01        0.01381338  0.32896352]
 [ 0.79648787  0.01        0.08860772  0.01        0.01        0.01
   0.01444472  0.02122173  0.07050229  0.01      ]
 [ 0.01        0.01        0.01        0.0555775   0.01        0.01160172
   0.91099381  0.01        0.01746489  0.01      ]
 [ 0.61711437  0.02337319  0.0

In [5]:
model.save('models/data_augmentation.h5')